In [20]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
#from assistant.graph import graph

from assistant.utils import load_pkl_files
from assistant.vectorstore import VectorStore
from assistant.configuration import Configuration

In [21]:
#Load the vectorstore and add documents --> TODO: clean this up
async def load_vectorstore():
    # Load the parsed document pkl files
    pkl_dict = load_pkl_files(parsed_document_dir="./parsed_documents")
    # Load the vectorstore
    vectorstore = VectorStore()
    vectorstore.add_pkl_dict_to_vectorstore(pkl_dict)
    return vectorstore
vectorstore = asyncio.run(load_vectorstore())

Collection 'in_memory_collection' created
 1. A_Method_for_Comparing_the_Fatigue_Performance_of_Forged, Adding to vector store...
 2. Characterization_of_forged_magnesium_alloys, Adding to vector store...
 3. Fatigue_of_Forged_AZ80_Magnesium_Alloy, Adding to vector store...
 4. Microstructure_and_texture_evolution_during_hot_deformation_of_AZ80_magnesium_alloy, Adding to vector store...
Time taken to add documents to vector store: 10.8 seconds


In [22]:
configurable = Configuration()

In [23]:
import json
from typing import List
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from assistant.prompts import document_grading_instructions

def document_search(search_query: str, configurable: Configuration):
    """
    Parses JSON string output from LLM grading response
    Args:
        state (dict): The current graph state

    Returns:
        list: List of parsed JSON grades
    """
    def _format_search_response(filtered_documents:List[Document]):
        formatted_documents = []
        for document in filtered_documents:
            formatted_documents.append({
                "title": document.metadata["filename"],
                "url": document.metadata["file_path"],
                "content": document.page_content,
                "context": document.metadata["context"]
            })
        return {
            "results": formatted_documents,
        }

    def _batch_instruction_template(docs:List[Document], search_query:str):
        return [document_grading_instructions.format(
            search_query=search_query,
            document=doc.page_content
        ) for doc in docs]

    llm_json_mode = ChatOpenAI(model=configurable.openai_model,
                               base_url=configurable.openai_base_url,
                               api_key=configurable.openai_api_key,
                               temperature=0.0,
                               response_format={"type": "json_object"})
    

    if not isinstance(search_query, str) or not search_query.strip():
        raise ValueError("Search query must be a non-empty string")

    # retrieve documents
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    retrieved_documents = retriever.invoke(search_query)

    # grade documents and parse JSON
    result_json = llm_json_mode.batch(
        _batch_instruction_template(retrieved_documents, search_query)
    )
    
    filtered_documents = []
    for i, response in enumerate(result_json):
        result = json.loads(response.content)
        if result['score'] == "yes":
            filtered_documents.append(retrieved_documents[i])

    return _format_search_response(filtered_documents)


In [24]:
retrieved_documents = document_search(search_query="How is texture related to the mechanical properties of magnesium alloys?", configurable=configurable)

c:\Users\TK\Desktop\Projects\Deep_Researcher_App\api\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3549: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [25]:
retrieved_documents

{'results': [{'title': 'Application of Machine Learning Modeling in Establishing the Process, Structure, and Property Relationships of the Cast-Forged AZ80 Magnesium Alloy',
   'url': 'data/documents/Application of Machine Learning Modeling in Establishing the Process, Structure, and Property Relationships of the Cast-Forged AZ80 Magnesium Alloy.pdf',
   'content': "| |3009 C|3505 C|4009 C| |\n|---|---|---|---|---|\n| | |100um|100um|100um|\n| | |20um|20um|20um|\n\nFigure 12: Microstructures of the samples compressed to a strain 1.0, at 0.001 s-1, and different temperatures. The top row shows lower magnification micrographs, while the bottom row shows the corresponding higher magnification micrographs [36].\n\nThe fracture behavior during the forging process is one of the main issues of this method. The cracks firstly initiate in the Mg/Mg17Al12 interface under the hoop tensile stress on equatorial free surface during deformation, and then it propagates along the grain boundaries [38]. 